In [17]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

## production Planning
An automobile assembly plant assembles two types of vehicles: a four-door saloon and a people carrier. Both vehicle types must pass through a painting plant and an assembly plant. If the painting plant only paints four-door saloons, it can paint some 2,000 vehicles each day, whereas if it paints only people carriers, it can paint some 1,500 vehicles each day. Moreover, if the assembly plant only assembles either four-door saloons or people carriers, it can assemble some 2,200 vehicles every day. Each people carrier implies an average profit of $3,000, whereas a fourdoor saloon implies an average profit of $2,100. maximise the vehicle assembly plant’s daily profit

In [20]:
#instatiate an object of ConcreteModel class
model = pyo.ConcreteModel()

# Sets
model.i = pyo.Set(initialize=range(1, 3))

# Parameters

# painting_parameter
model.pa = pyo.Param(model.i, initialize = {1:2000,2:1500})
pa = model.pa

# assembly_parameter
model.a = pyo.Param(model.i, initialize = {1:2200,2:2200})
a = model.a

#profit_parameter
model.p = pyo.Param(model.i, initialize = {1:2100,2:3000})
p = model.p

# Decision Variables

model.x = pyo.Var(model.i, within=pyo.NonNegativeReals)
x= model.x


#Objective Function
def daily_profit(model):
    return sum(p[i] * x[i] for i in model.i)


model.obj = pyo.Objective(rule=daily_profit, sense=pyo.maximize)

# Constraints
def painting_constraint(model,i):
    """fraction of painting for each car type must be less than or equal to 1

    Args:
        model (_type_): _description_
        i (_type_): _description_
    """
    return sum((1/pa[i])*x[i] for i in model.i) <= 1

def assembly_constraint(model,i):
    """fraction of assembly for each car type must be less than or equal to 1

    Args:
        model (_type_): _description_
        i (_type_): _description_
    """
    return sum((1/a[i])*x[i] for i in model.i) <= 1

model.const1 = pyo.Constraint(model.i, rule=painting_constraint)
model.const2 = pyo.Constraint(model.i, rule=assembly_constraint)

# Solve the model

solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(results)
print("PROFIT = ",model.obj())
for i in model.i:
    print("value of ", i," =",x[i]() )
    


Problem: 
- Name: unknown
  Lower bound: 4500000.0
  Upper bound: 4500000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Number of nonzeros: 8
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.002023458480834961
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

PROFIT =  4500000.0
value of  1  = 0.0
value of  2  = 1500.0
